# Q3 Using Scikit-Learn

In [1]:
import numpy as np
import pandas as pd
import time
import gc

from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
# Import statements run before running other code cells
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Classifier Setup

In [2]:
# XXX
# TODO: Read in all the data. Replace the 'xxx' with the path to the data set.
# XXX

def ifelse(cond,a, b):
    if cond:
        return a
    else:
        return b

data = pd.read_csv('pulsar_stars.csv')
# Separate out the x_data and y_data.
x_data = data.loc[:, data.columns != "y"]
y_data = data.loc[:, "y"]

# -------------------------------
# ADD CODE HERE
# -------------------------------

In [3]:
# The random state to use while splitting the data.
random_state = 100

# XXX
# TODO: Split 70% of the data into training and 30% into test sets. Call them x_train, x_test, y_train and y_test.
# Use the train_test_split method in sklearn with the parameter 'shuffle' set to true and the 'random_state' set to 100.
# XXX

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.70, test_size=0.30, random_state=random_state, shuffle=True)

# Linear Regression 

In [4]:
# XXX
# TODO: Create a LinearRegression classifier and train it.
# XXX

lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [5]:
# XXX
# TODO: Test its accuracy (on the training set) using the accuracy_score method.
# Note: Round the output values greater than or equal to 0.5 to 1 and those less than 0.5 to 0. You can use y_predict.round() or any other method.
# XXX

y_predict = lm.predict(X_train)
y_predict = [ifelse(y<=0.5, 0, 1) for y in y_predict]
np.round(accuracy_score(y_train, y_predict), 2)

0.97

In [6]:
# XXX
# TODO: Test its accuracy (on the testing set) using the accuracy_score method.
# Note: Round the output values greater than or equal to 0.5 to 1 and those less than 0.5 to 0. You can use y_predict.round() or any other method.
# XXX

y_predict = lm.predict(X_test)
y_predict = [ifelse(y<=0.5, 0, 1) for y in y_predict]
np.round(accuracy_score(y_test, y_predict), 2)

0.97

# Random Forest Classifier

In [7]:
# XXX
# TODO: Create a RandomForestClassifier and train it.
# WARNING: Ignore "FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22."10 in version 0.20 to 100 in 0.22."
# XXX

rf = RandomForestClassifier(random_state=random_state)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=100, verbose=0, warm_start=False)

In [8]:
# XXX
# TODO: Test its accuracy on the training set using the accuracy_score method.
# XXX

y_predict = rf.predict(X_train)
y_predict = [ifelse(y<=0.5, 0, 1) for y in y_predict]
np.round(accuracy_score(y_train, y_predict), 2)

1.0

In [9]:
# XXX
# TODO: Test its accuracy on the test set using the accuracy_score method.
# XXX

y_predict = rf.predict(X_test)
y_predict = [ifelse(y<=0.5, 0, 1) for y in y_predict]
np.round(accuracy_score(y_test, y_predict), 2)

0.98

## Feature Importance

In [10]:
# XXX
# TODO: Determine and print the feature importance as evaluated by the Random Forest Classifier.
# XXX

importances = rf.feature_importances_
print(importances)

[0.1257031  0.03131994 0.45697513 0.16258565 0.05165992 0.08903044
 0.05408017 0.02864564]


In [11]:
# XXX
# TODO: Sort them in the descending order and print the feature numbers[0 to ...].
#       Hint: There is a direct function available in sklearn to achieve this. Also checkout argsort() function in Python.
# XXX

print(np.argsort(importances)[::-1])

[2 3 0 5 6 4 1 7]


## Hyper-parameter Tuning

In [20]:
# XXX
# TODO: Tune the hyper-parameters 'n_estimators' and 'max_depth'.
# XXX

param_grid = { 
    'n_estimators': [80, 100, 120],
    'max_depth' : [6,8,10]
}
CV_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
CV_rf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=100, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [80, 100, 120], 'max_depth': [6, 8, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
# XXX
# TODO: Print the best params, using .best_params_
# XXX

print(CV_rf.best_params_)

{'max_depth': 10, 'n_estimators': 80}


In [22]:
# XXX
# TODO: Print the best score, using .best_score_.
# XXX

print(CV_rf.best_score_)

0.9793263090676884


# Support Vector Machine

## Pre-process

In [23]:
# XXX
# TODO: Pre-process the data to standardize or normalize it, otherwise the grid search will take much longer.
# Warning: Ignore "FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning"
# XXX

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [24]:
# XXX
# TODO: Create a SVC classifier and train it.
# XXX

svc = SVC(gamma='auto', random_state=random_state)
svc.fit(X_train_scaled, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=100, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
# XXX
# TODO: Test its accuracy on the training set using the accuracy_score method.
# XXX
# WARNING: Ignore "FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.""

y_predict = svc.predict(X_train_scaled)
y_predict = [ifelse(y<=0.5, 0, 1) for y in y_predict]
np.round(accuracy_score(y_train, y_predict), 2)

0.98

In [26]:
# XXX
# TODO: Test its accuracy on the test set using the accuracy_score method.
# XXX
# WARNING: Ignore "FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.""

X_test_scaled = scaler.transform(X_test)
y_predict = svc.predict(X_test_scaled)
y_predict = [ifelse(y<=0.5, 0, 1) for y in y_predict]
np.round(accuracy_score(y_test, y_predict), 2)

0.98

## Hyper-parameter Tuning

In [31]:
# XXX
# TODO: Tune the hyper-parameters 'C' and 'kernel' (use rbf and linear).
# XXX

param_grid = { 
    'C': [0.1, 1, 10],
    'kernel' : ['rbf', 'linear']
}
CV_svc = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5)
CV_svc.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=100, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1, 10], 'kernel': ['rbf', 'linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [32]:
# XXX
# TODO: Print the best score, using .best_score_.
# XXX

print(CV_svc.best_score_)

0.9788473818646233


In [34]:
# XXX
# TODO: Get the training and test set accuracy values after hyperparameter tuning.
# XXX

print(CV_svc.best_params_)
svc1 = SVC(C=10, kernel='rbf', gamma='auto', random_state=random_state)
svc1.fit(X_train_scaled, y_train)

{'C': 10, 'kernel': 'rbf'}


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=100, shrinking=True,
  tol=0.001, verbose=False)

In [35]:
# XXX
# TODO: Test its accuracy (on the training set) using the accuracy_score method.
# Note: Round the output values greater than or equal to 0.5 to 1 and those less than 0.5 to 0. You can use y_predict.round() or any other method.
# XXX

y_predict = svc1.predict(X_train_scaled)
y_predict = [ifelse(y<=0.5, 0, 1) for y in y_predict]
np.round(accuracy_score(y_train, y_predict), 2)

0.98

In [37]:
# XXX
# TODO: Test its accuracy (on the testing set) using the accuracy_score method.
# Note: Round the output values greater than or equal to 0.5 to 1 and those less than 0.5 to 0. You can use y_predict.round() or any other method.
# XXX

y_predict = svc1.predict(X_test_scaled)
y_predict = [ifelse(y<=0.5, 0, 1) for y in y_predict]
np.round(accuracy_score(y_test, y_predict), 2)

0.98

In [38]:
# XXX
# TODO: Calculate the rank test score, mean testing score and mean fit time for the 
# best combination of hyperparameter values that you obtained in Q3.2. The GridSearchCV 
# class holds a  ‘cv_results_’ dictionary that should help you report these metrics easily.
# XXX

CV_svc.cv_results_

{'mean_fit_time': array([0.16801162, 0.0775003 , 0.144732  , 0.11620269, 0.20466924,
        0.34949241]),
 'std_fit_time': array([0.01213826, 0.00133174, 0.00298388, 0.00717896, 0.00639229,
        0.03875908]),
 'mean_score_time': array([0.0278028 , 0.01045485, 0.02060361, 0.01004725, 0.01910968,
        0.00996866]),
 'std_score_time': array([0.00160598, 0.0002827 , 0.00046638, 0.00027586, 0.00042531,
        0.00020532]),
 'param_C': masked_array(data=[0.1, 0.1, 1, 1, 10, 10],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 0.1, 'kernel': 'rbf'},
  {'C': 0.1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'

In [40]:
# XXX
# TODO: Print the rank test score for the best combination of hyperparameter values that you obtained in Q3.2. The 
# GridSearchCV class holds a  ‘cv_results_’ dictionary that should help you report these metrics easily.
# XXX

print(CV_svc.cv_results_['rank_test_score'])

[6 5 3 3 1 2]


In [41]:
# XXX
# TODO: Print mean testing score for the best combination of hyperparameter values that you obtained in Q3.2. The 
# GridSearchCV class holds a  ‘cv_results_’ dictionary that should help you report these metrics easily.
# XXX

print(CV_svc.cv_results_['mean_test_score'])

[0.97573436 0.97717114 0.97780971 0.97780971 0.97884738 0.97796935]


In [42]:
# XXX
# TODO: Print mean fit time for the best combination of hyperparameter values that you obtained in Q3.2. The 
# GridSearchCV class holds a  ‘cv_results_’ dictionary that should help you report these metrics easily.
# XXX

print(CV_svc.cv_results_['mean_fit_time'])

[0.16801162 0.0775003  0.144732   0.11620269 0.20466924 0.34949241]


# PCA

In [166]:
# XXX
# TODO: Perform dimensionality reduction of the data using PCA.
#       Set parameters n_component to 8 and svd_solver to 'full'. Keep other parameters at their default value.
# XXX

pca = PCA(n_components=8, svd_solver='full', random_state=random_state)
pca.fit(x_data)  

PCA(copy=True, iterated_power='auto', n_components=8, random_state=100,
  svd_solver='full', tol=0.0, whiten=False)

In [167]:
# XXX
# TODO: Print Percentage of variance explained by each of the selected components
# XXX

print(pca.explained_variance_ratio_)

[8.71053041e-01 7.81934383e-02 4.11562290e-02 6.15967691e-03
 2.43717952e-03 9.58578490e-04 3.90570992e-05 2.79968662e-06]


In [168]:
# XXX
# TODO: The singular values corresponding to each of the selected components.
# XXX

print(pca.singular_values_)

[14430.28004546  4323.5214088   3136.68022725  1213.47665361
   763.30168073   478.70316467    96.62788002    25.87063984]
